In [1]:
import re
import pandas as pd;
import math
import numpy as np;
from operator import truediv

In [2]:
class Node:
    """ Node class for a decision tree. """
    def __init__(self, names):
        self.names = names

    def classify(x):
        """ Handled by the subclasses. """
        return None

    def dump(self, indent):
        """ Handled by the subclasses. """
        return None


class Leaf(Node):
    def __init__(self, names, value):
        Node.__init__(self, names)
        self.value = value

    def classify(self, x):
        return self.value

    def dump(self, indent):
        print(' %d' % self.value)


class Split(Node):
    def __init__(self, names, var, left, right):
        Node.__init__(self, names)
        self.var = var
        self.left = left
        self.right = right

    def classify(self, x):
        if x[self.var] == 0:
            return self.left.classify(x)
        else:
            return self.right.classify(x)
      
    def dump(self, indent):
        if indent > 0:
            print('')
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 0 :' % self.names[self.var],end='')
        self.left.dump(indent+1)
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 1 :' % self.names[self.var],end='')
        self.right.dump(indent+1)


Helper function computes entropy of Bernoulli distribution with parameter p

In [3]:
def entropy(p):
    # >>>> YOUR CODE GOES HERE <<<<
    # For now, always return "0":
    resultEntropy = []
    for value in p:
        if value.is_integer():
            resultEntropy.append(0.0)
        else:  
            Entropy = -1 *(value * math.log2(value) + (1-value) * math.log2(1-value))
            resultEntropy.append(Entropy)
    return resultEntropy;


Compute information gain for a particular split, given the counts 

py_pxi : number of occurences of y=1 with x_i=1 for all i=1 to n

pxi : number of occurrences of x_i=1

py : number of ocurrences of y=1


In [4]:
def infogain(py_pxi, pxi, py, total):
    # >>>> YOUR CODE GOES HERE <<<<
    # For now, always return "0":
    
    totalProbability = [py/total]
    LeftProbability = []
    RightProbability = []
    resultGain=[]
    totalEntropy = []
    LeftEntropy = []
    RightEntropy = []
    
    #finding the probability of left split values 
    for i,j in zip(py_pxi,pxi):
        if j == 0:
            LeftProbability.append(0.0)
        else:
            LeftProbability.append(i/j)
    
    #finding the probability of right split values 
    RightNumerator = [py-value for value in py_pxi]
    RightDenominator = [total-value for value in pxi]
    
    for i,j in zip(RightNumerator,RightDenominator):
        if j == 0:
            RightProbability.append(0.0)
        else:
            RightProbability.append(i/j)
            
    #finding the left entropy 
    LeftEntropy = entropy(LeftProbability)
    
    #finding the right entropy 
    RightEntropy = entropy(RightProbability) 
    
    #finding the total entropy 
    totalEntropy = entropy(totalProbability)

    #finding the Information Gain
    for i in range(len(pxi)-1):
        value = totalEntropy[0] - (pxi[i]/total)*LeftEntropy[i] - (1-pxi[i]/total)*RightEntropy[i]
        resultGain.append(value)

        
    return resultGain;


OTHER SUGGESTED HELPER FUNCTIONS:

-collect counts for each variable value with each class label

-find the best variable to split on, according to mutual information

-partition data based on a given variable	
	


In [5]:
def CountVariables(data, varnames,TargetColumn):
    # count total number of rows
    total = data.shape[0]
    InfoGain = {}
    py_pxi = []
    InformationGain = []
    
    #number of occurances of y=0
    py = (data.iloc[: , TargetColumn] == 0).sum(axis=0)
    
    #number of occurances of x=0
    pxi = (data == 0).sum(axis=0)
   
    # number of occurences of y=0 with x_i=0 for all i=1 to n
    for column in data.columns[:-1]:
        test = data.loc[(data[varnames[-1]] == 0) & (data[column] == 0)]
        py_pxi.append(test.shape[0])
    
    #compute information gain
    InformationGain = infogain(py_pxi, pxi, py, total)
    InfoGain = dict(zip(varnames, InformationGain))
    
    return InfoGain

In [6]:
# Load data from a file
def read_data(filename):
    f = open(filename, 'r')
    p = re.compile(',')
    data = []
    header = f.readline().strip()
    varnames = p.split(header)
    namehash = {}
    for l in f:
        data.append([int(x) for x in p.split(l.strip())])
    return (data, varnames)


Build tree in a top-down manner, selecting splits until we hit a pure leaf or all splits look bad.


In [7]:
def build_tree(data, varnames):
    # >>>> YOUR CODE GOES HERE <<<<
    # For now, always return a leaf predicting "1":
    InformationGain = {}
    TargetClass = -1
    data = pd.DataFrame(data)
    data.set_axis(varnames, axis=1, inplace=True)
    
    InformationGain = CountVariables(data,varnames,TargetClass)
    
    #Find the name of MaxInformation Gain column
    MaxInfoGainName = max(InformationGain, key=InformationGain.get)
    
    #Find the index of Max Information Gain
    MaxInfoGainIndex = varnames.index(MaxInfoGainName)
    
    #Find the value of Max Information Gain
    MaxInfoGainValue = InformationGain[MaxInfoGainName]
    
    #checking if feature at max information gain is zero
    
    if MaxInfoGainValue == 0.0:
        if data[varnames[-1]].unique()[0] == 0:
            return Leaf(varnames,0)
        else:
            return Leaf(varnames,1)
    
    else:
        #checking if all the values of Max Information column have same value.
        if len(data[MaxInfoGainName].unique()) == 1:
            return Leaf(varnames,data[MaxInfoGainName].unique()[0])
        
        
        else:
            #trim the data for all left subtree based on Max Info Gain Column
            left_Split_data = data.loc[data[MaxInfoGainName] == 0]
            
            #trim the data for all right subtree based on Max Info Gain Column
            right_Split_data = data.loc[data[MaxInfoGainName] == 1]
            
            #new_varnames = varnames
            #new_varnames.remove(MaxInfoGainName)
            
            #call the Split function recursively
            root = Split(varnames, MaxInfoGainIndex , build_tree(left_Split_data,varnames), build_tree(right_Split_data,varnames))

    return root


Here we load data.
Each example is a list of attribute values, where the last element in the list is the class value.


In [8]:
agaricus = ["agaricuslepiotatrain1.csv",
              "agaricuslepiotatest1.csv",
              "agaricuslepiotatest1.csv"]

dataset1 = ["data_sets1/training_set.csv",
            "data_sets1/validation_set.csv",
            "data_sets1/test_set.csv"]
            

dataset2 = ["data_sets2/training_set.csv",
            "data_sets2/validation_set.csv",
            "data_sets2/test_set.csv"]

# pick the dataset you want to use this time
dataset = agaricus
# dataset = dataset1

(train, varnames) = read_data(dataset[0])
(validation, validationvarnames) = read_data(dataset[1]) 
(test, testvarnames) = read_data(dataset[2]) 

In [9]:
pd.DataFrame(train).head()

,0,1,2,3,4,5,6,7,8,9,...,113,114,115,116,117,118,119,120,121,122
0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
1,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [10]:
#varnames

Build the decision tree

In [11]:
root = build_tree(train, varnames)

In [12]:
root.dump(0)



odor-foul = 0 :
| gill-size-broad = 0 :
| | odor-none = 0 :
| | | gill-spacing-close = 0 :
| | | | bruises?-bruises = 0 : 1
| | | | bruises?-bruises = 1 : 0
| | | gill-spacing-close = 1 : 1
| | odor-none = 1 :
| | | stalk-surface-above-ring-silky = 0 :
| | | | bruises?-bruises = 0 : 0
| | | | bruises?-bruises = 1 : 1
| | | stalk-surface-above-ring-silky = 1 : 1
| gill-size-broad = 1 :
| | spore-print-color-green = 0 : 0
| | spore-print-color-green = 1 : 1
odor-foul = 1 : 1


Calcuating the accuracy

In [13]:
def accuracy(data):
    correct = 0
    # The position of the class label is the last element in the list.
    yi = len(data[0]) - 1
    for x in data:
    # Classification is done recursively by the node class.
    # This should work as-is.
        pred = root.classify(x)
        if pred == x[yi]:
            correct += 1
        acc = float(correct)/len(data)
    return acc;
   

In [14]:
 print("Train Accuracy: {}".format(accuracy(train)))

Train Accuracy: 1.0


In [15]:
 print("Test Accuracy: {}".format(accuracy(test)))

Test Accuracy: 0.9792941176470589


In [16]:
# Computing with sklearn
# from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
# from sklearn.model_selection import train_test_split # Import train_test_split function
# from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
# col_names = varnames
# pima =  pd.DataFrame(test)
# pima.set_axis(col_names, axis=1, inplace=True)
# pima.head()
# X = pima[col_names]
# y = pima[col_names[-1]]

# # Split dataset into training set and test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

# clf = DecisionTreeClassifier(criterion="entropy")

# # Train Decision Tree Classifer
# clf = clf.fit(X_train,y_train)

# #Predict the response for test dataset
# y_pred = clf.predict(X_test)
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))